# **Homework 1: COVID-19 Cases Prediction (Regression)**

Author: Heng-Jui Chang

Slides: https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.pdf  
Video: TBA

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Get familiar with PyTorch.

If any questions, please contact the TAs via TA hours, NTU COOL, or email.
##
# 一个发现
- 通过增加Batch_Size ,kaggle 的 public 和 private 
    在0.94(train 95%),或0.95左右(train 90%),其余什么也不改
- Batch norm 结果很差


# **Download Data**


If the Google drive links are dead, you can download data from [kaggle](https://www.kaggle.com/c/ml2021spring-hw1/data), and upload data manually to the workspace.

In [12]:
tr_path = r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.train.csv'  # path to training data
tt_path = r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.test.csv'   # path to testing data

!gdown --id '19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF' --output covid.train.csv
!gdown --id '1CE240jLm2npU-tdz81-oVKEF3T2yfT1O' --output covid.test.csv

'gdown' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'gdown' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


# **Import Some Packages**

In [13]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data preprocess
import numpy as np
import csv
import os

# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

myseed = 42069  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# **Some Utilities**

You do not need to modify this part.

In [14]:
def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'

def plot_learning_curve(loss_record, title=''):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_steps = len(loss_record['train'])
    x_1 = range(total_steps)
    x_2 = x_1[::len(loss_record['train']) // len(loss_record['dev'])]
    figure(figsize=(6, 4))
    plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
    plt.plot(x_2, loss_record['dev'], c='tab:cyan', label='dev')
    plt.ylim(0.0, 5.)
    plt.xlabel('Training steps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()


def plot_pred(dv_set, model, device, lim=35., preds=None, targets=None):
    ''' Plot prediction of your DNN '''
    if preds is None or targets is None:
        model.eval()
        preds, targets = [], []
        for x, y in dv_set:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                preds.append(pred.detach().cpu())
                targets.append(y.detach().cpu())
        preds = torch.cat(preds, dim=0).numpy()
        targets = torch.cat(targets, dim=0).numpy()

    figure(figsize=(5, 5))
    plt.scatter(targets, preds, c='r', alpha=0.5)
    plt.plot([-0.2, lim], [-0.2, lim], c='b')
    plt.xlim(-0.2, lim)
    plt.ylim(-0.2, lim)
    plt.xlabel('ground truth value')
    plt.ylabel('predicted value')
    plt.title('Ground Truth v.s. Prediction')
    plt.show()

# **Preprocess**

We have three kinds of datasets:
* `train`: for training
* `dev`: for validation
* `test`: for testing (w/o target value)

## **Dataset**

The `COVID19Dataset` below does:
* read `.csv` files
* extract features
* split `covid.train.csv` into train/dev sets
* normalize features

Finishing `TODO` below might make you pass medium baseline.

In [15]:
from sklearn.model_selection import train_test_split

In [45]:
class COVID19Dataset(Dataset):
    ''' Dataset for loading and preprocessing the COVID19 dataset '''
    def __init__(self,
                 path,
                 mu,
                 std,
                 mode='train',
                 target_only=False):
        self.mode = mode

        # Read data into numpy arrays
        with open(path, 'r') as fp:
            data = list(csv.reader(fp))
            data = np.array(data[1:])[:, 1:].astype(float)
        
        if not target_only:
            feats = list(range(93))
#             feats=list(range(40))+[42, 43, 57, 60, 61, 75, 78, 79]
#             feats=list(range(40))+[42, 57, 60, 75, 78]
        else:
            # TODO: Using 40 states & 2 tested_positive features (indices = 57 & 75)
            pass

        if mode == 'test':
            # Testing data
            # data: 893 x 93 (40 states + day 1 (18) + day 2 (18) + day 3 (17))
            data = data[:, feats]
            self.data = torch.FloatTensor(data)
        else:
            # Training data (train/dev sets)
            # data: 2700 x 94 (40 states + day 1 (18) + day 2 (18) + day 3 (18))
            target = data[:, -1]
            data = data[:, feats]
            
            data_indices = list(range(len(data)))
            train_indices,val_indices=train_test_split(data_indices,test_size=0.01,
                                                      random_state=0)
            if mode=='train':
                indices=train_indices
            elif mode=='dev':
                indices=val_indices
                
            # Splitting training data into train & dev sets
#             if mode == 'train':
#                 indices = [i for i in range(len(data)) if i % 10 != 0]
#             elif mode == 'dev':
#                 indices = [i for i in range(len(data)) if i % 10 == 0]
            
            # Convert data into PyTorch tensors
            self.data = torch.FloatTensor(data[indices])
            self.target = torch.FloatTensor(target[indices])
        
        if mode=='train':
            self.mu=self.data[:,40:].mean(dim=0,keepdim=True)
            self.std=self.data[:,40:].std(dim=0,keepdim=True)
        else:
            self.mu=mu
            self.std=std
        self.data[:,40:]=(self.data[:,40:]-self.mu)/self.std

        # Normalize features (you may remove this part to see what will happen)
        
#         self.data[:, 40:] = \
#             (self.data[:, 40:] - self.data[:, 40:].mean(dim=0, keepdim=True)) \
#             / self.data[:, 40:].std(dim=0, keepdim=True)
#         if mode=='train':
#             mu=
        
    
    
        self.dim = self.data.shape[1]

        print('Finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'
              .format(mode, len(self.data), self.dim))

    def __getitem__(self, index):
        # Returns one sample at a time
        if self.mode in ['train', 'dev']:
            # For training
            return self.data[index], self.target[index]
        else:
            # For testing (no target)
            return self.data[index]

    def __len__(self):
        # Returns the size of the dataset
        return len(self.data)

## **DataLoader**

A `DataLoader` loads data from a given `Dataset` into batches.


In [46]:
def prep_dataloader(path,
                    mode,
                    batch_size,
                    mu,
                    std,
                    n_jobs=0,
                    target_only=False):
    ''' Generates a dataset, then is put into a dataloader. '''
    #     dataset = COVID19Dataset(path, mode=mode, target_only=target_only)  # Construct dataset
    dataset = COVID19Dataset(path, mu, std, mode=mode, target_only=target_only)
    if mode == 'train':
        mu = dataset.mu
        std = dataset.std
    dataloader = DataLoader(dataset,
                            batch_size,
                            shuffle=(mode == 'train'),
                            drop_last=False,
                            num_workers=n_jobs,
                            pin_memory=True)  # Construct dataloader
    return dataloader, mu, std

# **Deep Neural Network**

`NeuralNet` is an `nn.Module` designed for regression.
The DNN consists of 2 fully-connected layers with ReLU activation.
This module also included a function `cal_loss` for calculating loss.


In [47]:
class NeuralNet(nn.Module):
    ''' A simple fully-connected deep neural network '''
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()

        # Define your neural network here
        # TODO: How to modify this model to achieve better performance?
        self.net = nn.Sequential(
            nn.Linear(input_dim, 1)  ## wx modify
        )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        return self.net(x).squeeze(1)

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        # TODO: you may implement L2 regularization here
        return self.criterion(pred, target)

# **Train/Dev/Test**

## **Training**

In [30]:
def train(tr_set, dv_set, model, config, device):
    ''' DNN training '''

    n_epochs = config['n_epochs']  # Maximum number of epochs

    # Setup optimizer
    optimizer = getattr(torch.optim, config['optimizer'])(
        model.parameters(), **config['optim_hparas'])

    min_mse = 1000.
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0
    while epoch < n_epochs:
        model.train()                           # set model to training mode
        for x, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x, y = x.to(device), y.to(device)   # move data to device (cpu/cuda)
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
            mse_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            loss_record['train'].append(mse_loss.detach().cpu().item())

        # After each epoch, test your model on the validation (development) set.
        dev_mse = dev(dv_set, model, device)
        if dev_mse < min_mse:
            # Save model if your model improved
            min_mse = dev_mse
            print('Saving model (epoch = {:04d},train loss={:.4f} | loss = {:.4f})'
                .format(epoch + 1,loss_record['train'][-1] ,min_mse))
            torch.save(model.state_dict(), config['save_path'])  # Save model to specified path
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1

        epoch += 1
        loss_record['dev'].append(dev_mse)
        if early_stop_cnt > config['early_stop']:
            # Stop training if your model stops improving for "config['early_stop']" epochs.
            break

    print('Finished training after {} epochs'.format(epoch))
    return min_mse, loss_record

## **Validation**

In [31]:
def dev(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss

## **Testing**

In [32]:
def test(tt_set, model, device):
    model.eval()                                # set model to evalutation mode
    preds = []
    for x in tt_set:                            # iterate through the dataloader
        x = x.to(device)                        # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            preds.append(pred.detach().cpu())   # collect prediction
    preds = torch.cat(preds, dim=0).numpy()     # concatenate all predictions and convert to a numpy array
    return preds

# **Setup Hyper-parameters**

`config` contains hyper-parameters for training and the path to save your model.

In [33]:
device = get_device()                 # get the current available device ('cpu' or 'cuda')
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
target_only = False                   # TODO: Using 40 states & 2 tested_positive features

# TODO: How to tune these hyper-parameters to improve your model's performance?
config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 64 ,               # mini-batch size for dataloader
    'optimizer': 'Adam',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        'lr': 0.05,                 # learning rate of SGD
#         'momentum': 0.9              # momentum for SGD
    },
    'early_stop': 500,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.pth'  # your model will be saved here
}

# **Load data and model**

In [48]:
tr_set,tr_mu,tr_std= prep_dataloader(tr_path, 'train', config['batch_size'],None,None, target_only=target_only)
dv_set ,none_mu,none_std= prep_dataloader(tr_path, 'dev', config['batch_size'],tr_mu,tr_std, target_only=target_only)
tt_set,none_mu,none_std = prep_dataloader(tt_path, 'test', config['batch_size'],tr_mu,tr_std, target_only=target_only)

Finished reading the train set of COVID19 Dataset (2673 samples found, each dim = 93)
Finished reading the dev set of COVID19 Dataset (27 samples found, each dim = 93)
Finished reading the test set of COVID19 Dataset (893 samples found, each dim = 93)


In [35]:
class NeuralNet(nn.Module):
    ''' A simple fully-connected deep neural network '''
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()

        # Define your neural network here
        # TODO: How to modify this model to achieve better performance?
        self.net = nn.Sequential(
#             nn.BatchNorm1d(input_dim),
#             nn.Linear(input_dim, 86),
#             nn.ReLU(),
# #             nn.BatchNorm1d(64),
#             nn.Linear(86, 1)
            nn.Linear(input_dim,1)
        )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        return self.net(x).squeeze(1)

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        # TODO: you may implement L2 regularization here
        return self.criterion(pred, target)

In [36]:
  # Construct model and move to device

# **Start Training!**

In [37]:
device = get_device()                 # get the current available device ('cpu' or 'cuda')
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
target_only = False                   # TODO: Using 40 states & 2 tested_positive features

# TODO: How to tune these hyper-parameters to improve your model's performance?
# torch.optim.Adam()
config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 64 ,               # mini-batch size for dataloader
#     'optimizer': 'Adam',              # optimization algorithm (optimizer in torch.optim)
    'optimizer': 'Adam',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        'lr': 0.05,                 # learning rate of SGD
#         'momentum': 0.9              # momentum for SGD
    },
    'early_stop': 500,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.pth'  # your model will be saved here
}
model = NeuralNet(tr_set.dataset.dim).to(device)
model_loss, model_loss_record = train(tr_set, dv_set, model, config, device)

Saving model (epoch = 0001,train loss=201.3372 | loss = 187.5994)
Saving model (epoch = 0002,train loss=113.4743 | loss = 118.8651)
Saving model (epoch = 0003,train loss=74.3560 | loss = 73.6380)
Saving model (epoch = 0004,train loss=44.5977 | loss = 41.6325)
Saving model (epoch = 0005,train loss=25.2877 | loss = 23.0367)
Saving model (epoch = 0006,train loss=13.3109 | loss = 12.6103)
Saving model (epoch = 0007,train loss=6.1680 | loss = 6.5810)
Saving model (epoch = 0008,train loss=3.1044 | loss = 3.4594)
Saving model (epoch = 0009,train loss=1.7368 | loss = 2.0864)
Saving model (epoch = 0010,train loss=2.1520 | loss = 1.4715)
Saving model (epoch = 0011,train loss=1.1253 | loss = 1.2442)
Saving model (epoch = 0012,train loss=1.2634 | loss = 1.1444)
Saving model (epoch = 0013,train loss=1.0222 | loss = 1.1380)
Saving model (epoch = 0017,train loss=0.5709 | loss = 1.1277)
Saving model (epoch = 0018,train loss=0.9861 | loss = 1.1191)
Saving model (epoch = 0020,train loss=1.4022 | loss = 

Saving model (epoch = 0684,train loss=1.0344 | loss = 0.9176)

In [ ]:
plot_learning_curve(model_loss_record, title='deep model')

In [ ]:
del model
model = NeuralNet(tr_set.dataset.dim).to(device)
ckpt = torch.load(config['save_path'], map_location='cpu')  # Load your best model
model.load_state_dict(ckpt)
# plot_pred(dv_set, model, device)  # Show prediction on the validation set

In [49]:
model_chk=torch.load(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.pth')
model.load_state_dict(model_chk)
device='cpu'
train_mse=dev(tr_set, model, device)
valid_mse=dev(dv_set,model,device)
print('train mse %f, valid mse %f ' % (train_mse,valid_mse))


train mse 0.837004, valid mse 0.638302 


house price model 

In [39]:
def get_net(num_features):
    net=nn.Linear(num_features,1)
#     net = nn.Sequential(nn.Linear(num_features, 80), nn.ReLU(),
#                         nn.Linear(80, 1))
    for p in net.parameters():
        nn.init.normal_(p, mean=0, std=0.01)
    return net

In [41]:
net=get_net(93)
net_chk=torch.load(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.wx')
net.load_state_dict(net_chk)

In [42]:
def dev2(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    loss=nn.MSELoss(reduction='mean')
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            mse_loss = loss(pred,y)  # compute loss
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss

In [44]:
device='cpu'
train_mse=dev2(tr_set, net, device)
valid_mse=dev2(dv_set,net,device)
print('train mse %f, valid mse %f ' % (train_mse,valid_mse))

train mse 116.537081, valid mse 106.281395 


In [51]:
import pandas as pd

In [61]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.train.csv',)
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.test.csv')

In [62]:
train_data.iloc[:,40:-1]=train_data.iloc[:,40:-1].apply(lambda x : (x-x.mean()) / x.std())

In [63]:
train_features=torch.Tensor(train_data.iloc[:,1:-1].values)
train_labels=torch.Tensor(train_data.iloc[:,-1].values).view(-1,1)

In [64]:
train_features.shape

torch.Size([2700, 93])

In [65]:
train_labels.shape

torch.Size([2700, 1])

In [70]:
train_features[:5,40:]

tensor([[-0.4211, -0.5778, -0.4172, -0.3648, -0.9364,  1.3419,  0.9772,  1.7371,
          1.2597,  1.2648,  1.4035, -0.6428, -1.1475, -0.6367, -0.8175, -1.7962,
         -0.2355,  0.4302, -0.3703, -0.4991, -0.4239, -0.3698, -1.0681,  1.3591,
          1.0116,  1.7946,  1.2462,  1.2317,  1.3125, -0.6879, -1.2323, -0.6249,
         -0.9886, -1.8433, -0.1755,  0.4962, -0.2384, -0.3176, -0.3914, -0.3525,
         -1.0000,  1.2458,  0.9802,  1.6167,  1.2800,  1.1052,  1.2577, -0.7436,
         -1.1814, -0.5957, -0.9383, -1.7432, -0.1843],
        [-0.3631, -0.4919, -0.4138, -0.3602, -1.0552,  1.3435,  1.0012,  1.7768,
          1.2339,  1.2137,  1.3006, -0.6916, -1.2342, -0.6316, -0.9832, -1.8255,
         -0.1703,  0.5042, -0.2303, -0.3099, -0.3819, -0.3434, -0.9873,  1.2325,
          0.9691,  1.5993,  1.2680,  1.0884,  1.2460, -0.7470, -1.1833, -0.6020,
         -0.9332, -1.7253, -0.1791,  0.5688, -0.0598, -0.1345, -0.4252, -0.4103,
         -1.0570,  1.1106,  0.9776,  1.5261,  1.2571, 

In [66]:
dataset=torch.utils.data.TensorDataset(train_features,train_labels)
train_iter=torch.utils.data.DataLoader(dataset,64,shuffle=False)

In [67]:
model=model = NeuralNet(93)
model_chk=torch.load(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.pth')
model.load_state_dict(model_chk)

model.eval()
((model(train_features).view(-1,1).detach()-train_labels)**2).mean().item()

36.702796936035156

In [68]:
net=get_net(93)
net_chk=torch.load(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.wx')
net.load_state_dict(net_chk)
net.eval()
((net(train_features).view(-1,1).detach()-train_labels)**2).mean().item()

39.17701721191406

In [ ]:
 model.eval()                                # set model to evalutation mode
    total_loss = 0
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss


# ** modify by wx**

In [ ]:
# class NeuralNet(nn.Module):
#     ''' A simple fully-connected deep neural network '''
#     def __init__(self, input_dim):
#         super(NeuralNet, self).__init__()

#         # Define your neural network here
#         # TODO: How to modify this model to achieve better performance?
#         self.net = nn.Sequential(
# #             nn.BatchNorm1d(input_dim),
#             nn.Linear(input_dim, 64),
#             nn.ReLU(),
# #             nn.BatchNorm1d(64),
#             nn.Linear(64, 1)
#         )

#         # Mean squared error loss
#         self.criterion = nn.MSELoss(reduction='mean')

#     def forward(self, x):
#         ''' Given input of size (batch_size x input_dim), compute output of the network '''
#         return self.net(x).squeeze(1)

#     def cal_loss(self, pred, target):
#         ''' Calculate loss '''
#         # TODO: you may implement L2 regularization here
#         return self.criterion(pred, target)

# device = get_device()                 # get the current available device ('cpu' or 'cuda')
# os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
# target_only = False                   # TODO: Using 40 states & 2 tested_positive features

# # TODO: How to tune these hyper-parameters to improve your model's performance?
# config = {
#     'n_epochs': 3000,                # maximum number of epochs
#     'batch_size': 800,               # mini-batch size for dataloader
#     'optimizer': 'SGD',              # optimization algorithm (optimizer in torch.optim)
#     'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
#         'lr': 0.001,                 # learning rate of SGD
#         'momentum': 0.5             # momentum for SGD
#     },
#     'early_stop': 200,               # early stopping epochs (the number epochs since your model's last improvement)
#     'save_path': r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\models\model.pth'  # your model will be saved here
# }

# model = NeuralNet(tr_set.dataset.dim).to(device)  # Construct model and move to device

# model_loss, model_loss_record = train(tr_set, dv_set, model, config, device)
# plot_learning_curve(model_loss_record, title='deep model')
# del model
# model = NeuralNet(tr_set.dataset.dim).to(device)
# ckpt = torch.load(config['save_path'], map_location='cpu')  # Load your best model
# model.load_state_dict(ckpt)
# plot_pred(dv_set, model, device)  # Show prediction on the validation set

# **Testing**
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    print('Saving results to {}'.format(file))
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

preds = test(tt_set, model, device)  # predict COVID-19 cases with your model
save_pred(preds, r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\pred_new67.csv')         # save prediction file to pred.csv

# **Hints**

## **Simple Baseline**
* Run sample code

## **Medium Baseline**
* Feature selection: 40 states + 2 `tested_positive` (`TODO` in dataset)

## **Strong Baseline**
* Feature selection (what other features are useful?)
* DNN architecture (layers? dimension? activation function?)
* Training (mini-batch? optimizer? learning rate?)
* L2 regularization
* There are some mistakes in the sample code, can you find them?

# **Reference**
This code is completely written by Heng-Jui Chang @ NTUEE.  
Copying or reusing this code is required to specify the original author. 

E.g.  
Source: Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)
